In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Joe's cell
DRIVE_PATH = '/content/drive/MyDrive/Cal/Colab/DeepMusicVisualizer'

In [ ]:
# Richard's cell
DRIVE_PATH = 'TODO'

In [ ]:
SYM_PATH = '/content/DeepMusicVisualizer'
if not os.path.exists(SYM_PATH):
  !ln -s $DRIVE_PATH $SYM_PATH

In [ ]:
import sys
sys.path.append(SYM_PATH)

from osu_beatmap_parser.osureader.reader import BeatmapParser
from osu_beatmap_parser.osureader.beatmap import Beatmap
from osu_beatmap_parser.osureader.objects import HitObjectType
from data_manager.data_manager import DataManager as DM
from google.colab import files

%load_ext autoreload
%autoreload 2

In [ ]:
folder = os.path.join(SYM_PATH, 'data/beatmaps')
beatmaps, music_file = DM.get_beatmap(folder, True)

beatmap = beatmaps

print(music_file)

Available Songs: 
00) AfterlifeDabinRemix
01) Aeonian
02) BOOMBAYAH
03) BrainPower
04) Earthquake
05) Faded
06) JungleJuice
07) Astronomia
08) MarbleSoda
09) GiveItUp
10) Griztronics
11) Goodbye
12) MyLove
13) Nightlight
14) MonsterDotEXE
15) PopStars
16) Highscore
17) Stronger
18) SAD
19) TakeAHint
song selection(0-19):
5
Autoselected highest difficulty: HC's Hard6
/content/DeepMusicVisualizer/data/beatmaps/Faded/Faded - Alan Walker_ Iselin Solheim [MP3 128kbps].mp3


In [ ]:
timing_objects = beatmap.timing_objects
hit_objects = beatmap.hit_objects

res = []
for timing_object in timing_objects:
  if timing_object.uninherited == 1:
    res.append((timing_object.time, timing_object.beat_length))
print(res)

[(130, 666.666666666667), (32123, 666.666666666667), (54789, 666.666666666667), (152122, 666.666666666667), (194802, 666.666666666667), (201468, 789.473684210526), (202257, 833.333333333333), (202673, 1200.0), (203273, 1200.0)]


In [ ]:
max_time = hit_objects[-1].time + 5000
res = []
last_time = timing_objects[0].time
last_beatlen = timing_objects[0].beat_length
for timing_object in timing_objects:
  ind = (timing_object.time - last_time)/last_beatlen
  if timing_object.uninherited == 1:
    last_time = timing_object.time
    last_beatlen = timing_object.beat_length
    res.append(ind)
print(res)

[0.0, 47.98949999999998, 33.99899999999998, 145.99949999999993, 64.01999999999997, 9.998999999999995, 0.9994000000000004, 0.4992000000000002, 0.5]


In [ ]:
def to_frames(time):
  #time: time in ms
  # ms = (i * frame_duration)/sr * 1000
  return round(time / 1000 * sr / frame_duration)

def slider_duration(length, multiplier, beat_length):
  return length / (multiplier * 100) * beat_length

print(f'duration_ms:{(result_length * frame_duration/sr * 1000)}')

pulse = np.zeros(result_length)
motion = np.zeros(result_length)

PULSE_DURATION = 100 #ms
OSU_PULSE_REACT = 1.0
OSU_MOTION_REACT = 1.0
print(f'pulse_frames: {to_frames(PULSE_DURATION)}')
pulse_vec = np.ones(to_frames(PULSE_DURATION)) * OSU_PULSE_REACT
hit_objects = beatmap.hit_objects
timing_objects = beatmap.timing_objects

slider_multiplier = beatmap.difficult_settings.slider_multiplier
slider_multiplier_multiplier = 1
timing_object = timing_objects[0]

for hit_object in hit_objects:
  if len(timing_objects) > 0 and hit_object.time >= timing_objects[0].time:
    temp = timing_objects.pop(0)
    if temp.uninherited == 1:
      timing_object = temp
      slider_multiplier_multiplier = 1
    else:
      slider_multiplier_multiplier = 100/(-temp.beat_length)
  if HitObjectType.SLIDER in hit_object.type:
    start = to_frames(hit_object.time)
    motion_vec = OSU_MOTION_REACT * np.ones(to_frames(slider_duration(float(hit_object.length), slider_multiplier_multiplier * slider_multiplier, timing_object.beat_length)))
    print(f'motion_frames:{len(motion_vec)}')
    if start + len(motion_vec) > len(motion):
      print(f'last_time:{hit_object.time}')
      break;
    motion[start:start+len(motion_vec)] += motion_vec
  elif HitObjectType.CIRCLE in hit_object.type:
    start = to_frames(hit_object.time) - len(pulse_vec)//2
    if start+len(pulse_vec) > len(pulse):
      print(f'last_time:{hit_object.time}')
      break;
    pulse[start:start+len(pulse_vec)] += pulse_vec

return pulse, motion